### 5-1
#### 問
- 図5.7のグラフの最短路を求めよ

In [3]:
import networkx as nx

G = nx.DiGraph()
G.add_nodes_from(['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8'])
G.add_weighted_edges_from([('v1', 'v2', 20.0), ('v1', 'v3', 40.0), ('v1', 'v4', 58.0), 
                           ('v2', 'v4', 30.0), ('v2', 'v5', 51.0),
                           ('v3', 'v6', 16.0), ('v3', 'v7', 17.0),
                           ('v4', 'v5', 18.0), ('v4', 'v6', 7.0), ('v4', 'v7', 6.0),
                           ('v5', 'v8', 11.0), 
                           ('v6', 'v7', 4.0), ('v6', 'v8', 25.0), 
                           ('v7', 'v8', 24.0)])
print(nx.dijkstra_path(G, 'v1', 'v8'))

['v1', 'v2', 'v4', 'v5', 'v8']


### 5-2
#### 問
- クラスカル・プリムのアルゴリズムの正当性を示せ

#### 答
##### クラスカル法
- クラスカル法のループ内で $T \cup \{e_k\}$ が閉路を持たない時, 辺を小さい順にソートしていることから辺$e_k$ は $cut(T, V-T)$の中で最小の辺である.
- 定理5.1より, 任意のカット$cut(S, V-S)$に含まれる辺のうち, 重み最小のものは最小全域木に含まれることから, $e_k$は最小全域木に含まれる.
- よって, クラスカル法は最小全域木に含まれるような辺を常に採用して, 全域木を構成することから最小全域木を返す.

##### プリム法
- 定理5.1より, プリム法のループ内で採用される$e$は最小全域木に含まれるものであり、そのような辺をのみ採用して全域木を構成することから, プリム法は最小全域木を返す.

### 5-3
#### 問
- 全域木のうち重みがk番目に小さいものをk-最小全域木とする. 2-最小全域木を求めるアルゴリズムを構成せよ


In [5]:
class UnionFind():
    def __init__(self, n) -> None:
        self.par = [-1] * n
        self.siz = [1] * n

    def root(self, x):
        if self.par[x] == -1:
            return x
        else:
            self.par[x] = self.root(self.par[x])
            return self.par[x]

    def issame(self, x, y):
        return self.root(x) == self.root(y)

    def unite(self, x, y):
        x = self.root(x)
        y = self.root(y)

        if x == y:
            return False

        if self.siz[x] < self.siz[y]:
            x, y = y, x

        self.par[y] = x
        self.siz[x] += self.siz[y]
        return True

    def size(self, x):
        return self.siz[self.root(x)]

In [27]:
# O(|V||E|log(|V|))
# 図5-10
N = 6 # #node 
M = 10 # #edge
edge = [[0, 1, 5], [0, 2, 7], [1, 2, 6], [1, 3, 3], [1, 4, 4], [2, 3, 8], [2, 4, 2], [3, 4, 1], [3, 5, 10], [4, 5, 9]] # [node, node, cost]

def findSecondSpanningTree(N, M, edge):
    sort_edge = sorted(edge, key=lambda x:x[2])
    minSpanningTree = kruskal(N, sort_edge)[1]
    
    INF = float('inf')
    secondSpanningTree = [None, INF]
    for edge in minSpanningTree:
        print("~~~~~~~~")
        print(*edge)
        res = kruskal(N, sort_edge, edge)
        if res[0] and secondSpanningTree[1] > res[2]:
                secondSpanningTree =  [res[1], res[2]]
    
    print("~~~~~~~~")
    print("ans")
    print(*secondSpanningTree[0])
    print(secondSpanningTree[1])
    
    
def kruskal(N, sort_edge, excludeEdge = None):     
    uf = UnionFind(N)
    spanningTree = []
    totalCost = 0
    for a, b, c in sort_edge:
        if excludeEdge != None:
            if a == excludeEdge[0] and b == excludeEdge[1]:
                continue
        if uf.issame(a, b):
            continue
        else:
            uf.unite(a, b)
            spanningTree.append([a, b])
            totalCost += c
    
    print(uf.size(0))
    print(*spanningTree)
    print(totalCost)
    return [uf.size(0) == N, spanningTree , totalCost]

findSecondSpanningTree(N, M, edge)

6
[3, 4] [2, 4] [1, 3] [0, 1] [4, 5]
20
~~~~~~~~
3 4
6
[2, 4] [1, 3] [1, 4] [0, 1] [4, 5]
23
~~~~~~~~
2 4
6
[3, 4] [1, 3] [0, 1] [1, 2] [4, 5]
24
~~~~~~~~
1 3
6
[3, 4] [2, 4] [1, 4] [0, 1] [4, 5]
21
~~~~~~~~
0 1
6
[3, 4] [2, 4] [1, 3] [0, 2] [4, 5]
22
~~~~~~~~
4 5
6
[3, 4] [2, 4] [1, 3] [0, 1] [3, 5]
21
~~~~~~~~
ans
[3, 4] [2, 4] [1, 4] [0, 1] [4, 5]
21


### 5-4
#### 問
- 図5-23のように与えられたデータ点に対し、単リンク法によるクラスタリングを行いデンドログラムをかけ
#### 答


In [30]:
N = 5
M = 10
edge = [[0, 1, 8], [0, 2, 3], [0, 3, 6], [0, 4, 1], [1, 2, 7], [1, 3, 10], [1, 4, 9], [2, 3, 2], [2, 4, 4], [3, 4, 5]]

def kruskal(N, edge):   
    sort_edge = sorted(edge, key=lambda x:x[2])
    uf = UnionFind(N)
    for a, b, c in sort_edge:
        if uf.issame(a, b):
            continue
        else:
            uf.unite(a, b)
            print("~~~~")
            print(a)
            print(b)

kruskal(N, edge)

~~~~
0
4
~~~~
2
3
~~~~
0
2
~~~~
1
2


### 5-5
#### 問
- 最小費用流問題を解くことで輸送問題を解く方法を考えよ
#### 答
- Sのnodeにコスト0, 容量無限大のedgeをsから張る
- Cのnodeにコスト0, 容量無限大のedgeをtへ張る.
- この時, sからtへの最小費用流問題を解くと, それは輸送問題の解となる.

### 5-6
#### 問
- 2部グラフのマッチングで辺の数が最大のものを求める問題を最大マッチング問題と呼ぶ.
- 最大流問題を利用することで最大マッチング問題を解く方法を考えろ

#### 答
- 片側のnodeに容量1のedgeをsから張る.
- もう片側のnodeに容量1のedgeをtへ張る
- node間のedgeも容量1とする.
- この状態で, sからtへの最大流問題を解くとフローの大きさが最大マッチングのマッチ数となる.